In [7]:
import vk_api
import json
from pandas.io.json import json_normalize
import pandas as pd
import datetime

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
import time
import warnings
warnings.filterwarnings("ignore")

Задача предсказать возраст

метрика MSE

In [2]:
vk_session = vk_api.VkApi('', '')
vk_session.auth()



In [3]:
vk_api = vk_session.get_api()

In [155]:
#posts = getLikes (5117073,1, vk_api)

In [4]:
def get_friends(userId, api):
    user_friends = api.friends.get(user_id=userId, order='hints', fields = 'id, nickname, domain, sex, bdate, city, country, timezone, has_mobile, contacts, education, online, relation,  status, universities , faculty, faculty_name, relation') # возвращает в том порядке, как расположены в разделе Мои
    return user_friends

In [39]:
def get_posts(userId, api):
    user_posts = api.wall.get(owner_id=userId, filter='owner', count = 100) 
    return user_posts

In [42]:
def transformDataToDf(data, file_txt, file_csv):
    with open(file_txt,'w', encoding='utf-8') as file:
    json.dump(posts,file, sort_keys = True, indent = 4, ensure_ascii=False)
        
    df = pd.read_json(file_txt, encoding='utf-8')
    df2 = json_normalize(df['items'])
    df2.to_csv(file_csv,sep = ',')
    
    df = pd.read_csv(file_csv,  sep=',')
    
    return df

### Получение данных по друзьям

In [126]:
friends =  get_friends(5117073,vk_api)

In [127]:
df_f = transformDataToDf(friends, 'c:/friends.txt', 'c:/friends.csv')

In [128]:
df_f = df_f.drop(df_f[df_f['deactivated']=='deleted'].index)
df_f = df_f.drop(df_f[df_f['deactivated'] =='banned'].index)
df_f.shape

(122, 35)

In [129]:
#Определяем возраст человека, если не указан ставим 0
df_f['bdate'] = df_f['bdate'].astype('str')
df_f['bdate']= df_f['bdate'].apply(lambda x: '01.01.2019' if len(x) < 8 else x)
df_f['age'] = df_f.bdate.str[-4:]
df_f['age'] = df_f['age'].astype('int')
tday = datetime.datetime.now()
df_f['age2'] = tday.year
df_f['age2'].astype('int')
df_f['age3'] = df_f['age2'] - df_f['age']




In [130]:
cols = ['id','first_name','last_name','graduation', 'city.id','city.title','country.id','country.title','education_form','education_status','faculty','faculty_name','relation','relation_partner.id',
'sex','status','bdate','age3']
df_f = df_f[cols]

In [131]:
df_f[df_f['id']==6194676]

,id,first_name,last_name,graduation,city.id,city.title,country.id,country.title,education_form,education_status,faculty,faculty_name,relation,relation_partner.id,sex,status,bdate,age3


In [133]:
df_f.head()

,id,first_name,last_name,graduation,city.id,city.title,country.id,country.title,education_form,education_status,faculty,faculty_name,relation,relation_partner.id,sex,status,bdate,age3
0,88111,Анастасия,Ахметзянова,0.0,2.0,Санкт-Петербург,1.0,Россия,NaN,NaN,0.0,NaN,3.0,NaN,1,All you need is love,01.01.2019,0
1,3567843,Константин,Стрельцов,2004.0,284.0,Караганда,4.0,Казахстан,Очное отделение,NaN,9918.0,Факультет инновационных технологий (Информацио...,4.0,NaN,2,NaN,19.7.1982,37
2,9067562,Дарья,Баскакова,0.0,99.0,Новосибирск,1.0,Россия,NaN,NaN,0.0,NaN,4.0,1663884.0,1,NaN,01.01.2019,0
3,7862738,Ольга,Стрельцова,0.0,284.0,Караганда,4.0,Казахстан,NaN,NaN,0.0,NaN,4.0,NaN,1,Happiness is the truth,11.1.1983,36
4,202788920,Margo,Almatyfashion,0.0,183.0,Алматы,4.0,Казахстан,NaN,NaN,0.0,NaN,0.0,NaN,1,Тел 87078143309,01.01.2019,0


### Получение данных - посты
собираем в цикле посты каждого друга

In [106]:
users_list = df_f.id


In [18]:
#df = df[df['text'].notnull()]
#cols = ['owner_id','text','comments.count','date','post_source.platform','likes.count','reposts.count']
#df = df[cols]

In [81]:
posts =  get_posts(88111,vk_api)

In [ ]:
#
df_all = pd.DataFrame()
for user_id in users_list:
    posts =  get_posts(user_id,vk_api)
    df = transformDataToDf(posts, 'c:/data.json', 'c:/data.csv')
    if df.shape[0] > 0  :
        df = df[df['text'].notnull()]
        df['text'] = df['text'].astype('str')
        df = df[df['text'].str.len()>100]
        cols = ['owner_id','text','comments.count','date','likes.count','reposts.count']
        df = df[cols]
    
        df_all = df_all.append(df)
        

In [121]:
#выгружаем в файл для повторного использования без обращения к вк
df_all.to_csv('c:/data_all.csv',sep = ',', encoding='utf-8')

In [122]:
#считаваем из файла, чтобы не обращатья повторно к вк (после первого прохода, запускаем только получение из файла)
df_a = pd.read_csv('c:/data_all.csv',  sep=',')

In [134]:
#объединяем данные с постов с информацией о пользователе
df_u = pd.merge(df_a,df_f, how = 'inner',left_on="owner_id" , right_on="id" )

In [135]:
df_val = df_u[df_u['age3']==0] #убираем 
df_u = df_u[df_u['age3']>0]





In [150]:
df_u.shape, df_f.shape

((583, 26), (122, 19))

In [ ]:
#выгружаем в файл для повторного использования без обращения к вк
df_u.to_csv('c:/data_all_union.csv',sep = ',', encoding='utf-8')
df_f.to_csv('c:/data_friends.csv',sep = ',', encoding='utf-8')

###  Загрузка данных из файла(без подключения к ВК)


In [762]:
df_f = pd.read_csv('c:/data_friends.csv',  sep=',')
df_u = pd.read_csv('c:/data_all_union.csv',  sep=',')

### Обработка текста

In [763]:
text = df_u['text']

In [764]:
import re
import pymorphy2 
morph = pymorphy2.MorphAnalyzer()

Добалена лемматизация из библиотеки pymorphy2, так же регулярными выражениями удалила нетекстовые символы, ссылки и слова содержащие цирфы(тм было много мусора)

In [768]:
from nltk.corpus import stopwords 
from nltk.stem.wordnet import WordNetLemmatizer
import string

stop = set(stopwords.words('russian'))
exclude = set(string.punctuation) 



def clean(doc):
    #print(doc)
    doc = str(doc)
    stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
    punc_free = ''.join([ch for ch in stop_free if ch not in exclude])
    #normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
    normalized = " ".join(morph.parse(word)[0].normal_form for word in punc_free.split())
    clean = re.sub("\s\W*\s|\w*\d\w*|http\w*"," ",normalized) 

    return clean





In [769]:
doc_clean = [clean(doc) for doc in df_u['text']]   


In [770]:
#исходный текст
df_u['text'][582] 

'Детская ванночка, стул для кормления из Икеи красный, вкладка для новорожденного в эрго-рюкзак, накидка на ребенка для переноски в рюкзаке, детские вешалки, немного еды в банках. Кому?'

In [771]:
#строка после обработки
doc_clean[582]

'детский ванночка стул кормление икея красный вкладка новорождённый эргорюкзак накидка ребёнок переноска рюкзак детский вешалка немного еда банка кома'

In [772]:
len(doc_clean)

583

In [775]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(min_df=0.002, max_df=1)


In [776]:
features = tf_vectorizer.fit_transform(doc_clean).todense()
features


matrix([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]])

In [778]:
from sklearn.feature_extraction.text import CountVectorizer
count_vectorizer = CountVectorizer(min_df=0.002, max_df=1.0)
count_features = count_vectorizer.fit_transform(doc_clean).todense()
count_features

matrix([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [822]:
features.shape, count_features.shape

((583, 2788), (583, 2788))

In [779]:
features.shape, count_features.shape
tf_vectorizer.get_feature_names()[2773], count_vectorizer.get_feature_names()[2773]

('это', 'это')

In [780]:
tokens = pd.DataFrame(count_features)
tf_tokens = pd.DataFrame(features)

In [781]:
tokens.head()

,0,1,2,3,4,5,6,7,8,9,...,2778,2779,2780,2781,2782,2783,2784,2785,2786,2787
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [782]:
# Находим количественное вхождение каждого слова:
matrix_freq = np.asarray(count_features.sum(axis=0)).ravel()
final_matrix = np.array([np.array(count_vectorizer.get_feature_names()), matrix_freq])

In [783]:
final_matrix

array([['air', 'airdance', 'airdancespb', ..., 'январь', 'яркий', 'ёлка'],
       ['2', '2', '2', ..., '9', '8', '4']], dtype='<U63')

In [790]:
#Выбираем наиболее частотные слова
final_matrix.T
fm = pd.DataFrame(final_matrix.T)
fm.columns = ['token', 'cnt']
fm['cnt'] = fm['cnt'].astype(int)
fm = fm.sort_values(by='cnt', ascending=False)


In [793]:
fm = fm[fm['cnt']>90]
#fm.token.tolist()

In [794]:
fm.index.tolist()

[2773,
 126,
 194,
 340,
 2341,
 603,
 3,
 560,
 1018,
 2012,
 1534,
 2126,
 533,
 128,
 1303,
 129,
 134,
 133,
 101]

In [795]:
fm

,token,cnt
2773,это,248
126,triadadance,189
194,алматы,186
340,весь,154
2341,студия,137
603,день,127
3,almaty,116
560,группа,116
1018,который,116
2012,ребёнок,115


In [533]:
tokens = tokens.drop(columns=fm.index.tolist())


In [703]:
tf_tokens = tf_tokens.drop(columns=fm.index.tolist())

In [253]:
#####################

In [20]:
#import gensim
#from gensim import corpora

# Creating the term dictionary of our courpus, where every unique term is assigned an index. 
#dictionary = corpora.Dictionary(doc_clean)

# Converting list of documents (corpus) into Document Term Matrix using dictionary prepared above.
#doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]

In [806]:
y = df_u['age3']
X = tf_tokens

In [798]:
y.shape, X.shape

((583,), (583, 2788))

Попробуем линейную регрессию как baseline


In [799]:
from sklearn import cross_validation, tree, linear_model
from sklearn.linear_model import LinearRegression
from sklearn.metrics import explained_variance_score
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import RandomizedSearchCV
regr = linear_model.LinearRegression()

In [171]:
#df_u['tokens'] = df_u['text'].apply(lambda x: clean(x).split())


In [172]:
#df_u['vector'] = df_u['tokens'].apply(lambda x: dictionary.doc2bow(x))

In [ ]:
#cols = ['id', 'city.id','country.id','relation','sex','graduation']
#X = df_u[cols]
#y = df_u['age3']

In [800]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [801]:
regr.fit(X_train, y_train)
y_pred = regr.predict(X_test)

In [802]:
mse = mean_squared_error(y_test, y_pred)
print("MSE: %.2f " % mse)

MSE: 381.34 


In [803]:
from sklearn.linear_model import Ridge

clf = Ridge(alpha=1.0, random_state=42)
clf.fit(X_train, y_train) 

Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=42, solver='auto', tol=0.001)

In [804]:
y_pred = clf.predict(X_test)

In [805]:
mse = mean_squared_error(y_test, y_pred)
print("MSE: %.2f " % mse)

MSE: 392.71 


  MSE было 496 стало 392

In [807]:
def randomized_cv(model, param_grid, x_train=X_train, y_train=y_train):
    grid_search = RandomizedSearchCV(model, param_grid, cv=5, scoring='mean_squared_error', n_iter =9)
    t_start = time.time()
    grid_search.fit(x_train, y_train)
    t_end = time.time()
    print('model {} best mse is {}'.format(model.__class__.__name__, grid_search.best_score_))
    print('time for training is {} seconds'.format(t_end - t_start))
    print(grid_search.best_score_)
    return grid_search.best_estimator_

In [816]:
rdg = Ridge(alpha=1.0)
param_grid = {
    'alpha': np.arange(0,1.0,0.01),
    'max_iter': range(100,10000,10)
}
grid = randomized_cv(rdg, param_grid)

model Ridge best mse is -246.27145634596934
time for training is 1.8751075267791748 seconds
-246.27145634596934


In [817]:
y_pred = grid.predict(X_test) 

In [818]:
mse = mean_squared_error(y_test, y_pred)
print("MSE: %.2f " % mse)

MSE: 367.48 


In [819]:
from sklearn.linear_model import Lasso
lasso = Lasso()
param_grid = {
    'alpha': np.arange(0,1.0,0.01),
    'max_iter': range(100,10000,10)
}
grid = randomized_cv(lasso, param_grid)

model Lasso best mse is -316.5242438916712
time for training is 4.934282302856445 seconds
-316.5242438916712


In [820]:
y_pred = grid.predict(X_test) 

In [821]:
mse = mean_squared_error(y_test, y_pred)
print("MSE: %.2f " % mse)

MSE: 414.45 


Ridge показывает результат лучше

In [824]:
from spellchecker import SpellChecker
spell = SpellChecker()

In [826]:
misspelled = spell.unknown(['гулять', 'поддем', 'парк','завтра'])
for word in misspelled:
    print(spell.correction(word))
    print(spell.candidates(word))

поддем
{'поддем'}
парк
{'парк'}
завтра
{'завтра'}
гулять
{'гулять'}
